# Enforcing Interfaces with Elegance

## Polymorphism

One interface, many behaviors

In [1]:
class Trader:
    def execute(self):
        print("Executing manual trade")

class AlgoTrader(Trader):
    def execute(self):
        print("Executing algorithmic trade")

class HighFrequencyTrader(Trader):
    def execute(self):
        print("Executing HFT")

traders = [Trader(), AlgoTrader(), HighFrequencyTrader()]
for t in traders:
    t.execute()

Executing manual trade
Executing algorithmic trade
Executing HFT


## What Is an Abstract Base Class (ABC)?

- A class that **cannot be instantiated** directly
- Serves as a **blueprint** for other classes
- Defines **abstract methods** that **must be implemented** by subclasses
- Ensures a consistent interface across related classes

## Code Example

In [2]:
from abc import ABC, abstractmethod

class Trader(ABC):
    def __init__(self, name):
        self.name = name

    @abstractmethod
    def execute_trade(self, symbol, quantity):
        pass

**Trader** is now abstract — you **cannot create a Trader** object directly. Any subclass **must implement execute_trade()**

In [3]:
class AlgoTrader(Trader):
    def execute_trade(self, symbol, quantity):
        print(f"{self.name} executes algorithmic trade: {symbol}, {quantity}")

## Why Use ABCs?

- Enforce a **contract**: all subclasses must implement key methods
- Catch missing implementations **early**
- Improve **code reliability**, especially in large systems

## What Is Operator Overloading?

Allows you to customize the behavior of standard operators (+, -, ==, etc.) for your own classes

Achieved by defining special methods (also called dunder methods) like `__add__`, `__eq__`, `__lt__`, etc.

Makes your objects more intuitive, readable, and expressive

Operator to Method Mapping

Operator Method Description: + `__add__` Addition - `__sub__` Subtraction == `__eq__` Equality check < `__lt__` Less than > __gt__ Greater than

In [4]:
class Order:
    def __init__(self, symbol, quantity):
        self.symbol = symbol
        self.quantity = quantity

    def __add__(self, other):
        if self.symbol == other.symbol:
            return Order(self.symbol,
                        self.quantity + other.quantity)
        raise ValueError("Cannot add orders with different symbols")

    def __str__(self):
        return f"{self.quantity} shares of {self.symbol}"

In [5]:
o1 = Order("AAPL", 100)
o2 = Order("AAPL", 50)
o3 = o1 + o2
print(o3)  # Output: 150 shares of AAPL

150 shares of AAPL


## Object Representation

Why Customize Object Representation?

By default, printing an object shows a generic memory address

You can redefine how your object appears using special methods like `__str__` and `__repr__`

This improves readability, debugging, and logging

In [6]:
class Order:
    def __init__(self, symbol, quantity, price):
        self.symbol = symbol
        self.quantity = quantity
        self.price = price

    def __str__(self):
        return f"{self.quantity} shares of {self.symbol} at ${self.price:.2f}"

    def __repr__(self):
        return f"Order('{self.symbol}', {self.quantity}, {self.price})"

In [7]:
o = Order("AAPL", 100, 150.75)
print(o)   # Output: 100 shares of AAPL at $150.75
repr(o)    # Output: Order('AAPL', 100, 150.75)

100 shares of AAPL at $150.75


"Order('AAPL', 100, 150.75)"

## @property

What Is @property?

- A built-in Python decorator that turns a method into an attribute-like interface
- Allows you to define getters, setters, and deleters
- Supports encapsulation without sacrificing readability

Why Use It?

- Protect internal data with validation logic
- Keep method calls looking like simple attribute access
- Avoid verbose get_ and set_ methods

In [8]:
class Temperature:
    def __init__(self, celsius):
        self._celsius = celsius

    @property
    def fahrenheit(self):
        return (self._celsius * 9/5) + 32

    @fahrenheit.setter
    def fahrenheit(self, value):
        self._celsius = (value - 32) * 5/9

    @fahrenheit.deleter
    def fahrenheit(self):
        print("Fahrenheit property deleted")
        del self._celsius

In [9]:
t = Temperature(0)
print(t.fahrenheit)        # 32.0
t.fahrenheit = 212
print(t._celsius)          # 100.0

del t.fahrenheit           # Output: Fahrenheit property deleted

32.0
100.0
Fahrenheit property deleted


## Exception

In [10]:
class TradeError(Exception):
    pass

def execute_trade(qty):
    if qty <= 0:
        raise TradeError("Quantity must be positive")

In [11]:
try:
    execute_trade(-10)
except TradeError as e:
    print(f"Trade failed: {e}")
else:
    print("Trade executed successfully")
finally:
    print("Cleaning up resources")

Trade failed: Quantity must be positive
Cleaning up resources


## Exception Handling Best Practices

Keep `try` blocks **laser-focused on the specific operation that may fail**

Catch only **specific exceptions**; avoid bare `except:` clauses

Use `else:` for code when no exception occurs and `finally:` for guaranteed cleanup

Chain exceptions with `raise NewError from original_error` to preserve context

Create custom exceptions by subclassing `Exception` for domain-specific errors

In [15]:
# Chain exceptions
import requests

def fetch_data(url):
    try:
        response = requests.get(url)
    except requests.ConnectionError as e:
        raise ValueError("Failed to fetch data") from e

try:
    fetch_data("https://invalid.example.com")
except ValueError as e:
    print(e)  # ValueError: Failed to fetch data
    print(e.__cause__)  # ConnectionError: original error details

Failed to fetch data
HTTPSConnectionPool(host='invalid.example.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000150CA6E9220>: Failed to resolve 'invalid.example.com' ([Errno 11001] getaddrinfo failed)"))


`e.__cause__` is what makes chain exception useful